In [ ]:
## FILL IN YOUR NAME
NAME = "example"

In [ ]:
import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

%load_ext dotenv
%dotenv

import mlflow.keras

## Configure Experiment Name

In [ ]:
# configure experiment
experiment_name = f"keras-{NAME}"
mlflow.set_experiment(experiment_name)

## Autolog

In this example we will show how to automatically log parameters, metrics and artifacts with MLflow. We just need to add **one line of code** to enable the automatic logging.

The following libraries support autologging:

* Scikit-learn
* TensorFlow and Keras
* Gluon
* XGBoost
* LightGBM
* Statsmodels
* Spark
* Fastai
* Pytorch (using [Pytorch Lighting](https://pytorch-lightning.readthedocs.io/en/latest/))


## Reuters Topic Classification

In this example, we will be working with the Reuters dataset, a set of short newswires and their topics, published by Reuters in 1986. It's a very simple, widely used toy dataset for text classification. There are 46 different topics; some topics are more represented than others, but each topic has at least 10 examples in the training set. 

More info about this dataset can be found [here](https://keras.io/api/datasets/reuters/).

In [ ]:
# enable automatic logging
mlflow.keras.autolog()

max_words = 1000
batch_size = 32
epochs = 5

print("Loading data...")
(x_train, y_train), (x_test, y_test) = \
    reuters.load_data(num_words=max_words, test_split=0.2)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

num_classes = np.max(y_train) + 1
print(num_classes, "classes")

print("Vectorizing sequence data...")
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode="binary")
x_test = tokenizer.sequences_to_matrix(x_test, mode="binary")
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

print("Convert class vector to binary class matrix " 
      "(for use with categorical_crossentropy)")
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

print("Building model...")
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

history = model.fit(
    x_train, y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    verbose=1, 
    validation_split=0.1
)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])

## Tracking Server

Let's check the experiment logs in the tracking server!